## 0. Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import time
import pandas as pd
import csv
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , recall_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pickle

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## 1. Detections

In [4]:
cap = cv2.VideoCapture(0)

# model initiation
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        # Feed color conversion
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # pose detections
        results = pose.process(image)
        
        # revert back to bgr for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        cv2.imshow('Raw Feed',image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    

## 1.1 Video for Training

In [5]:
cap =cv2.VideoCapture(0)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
vwriter = cv2.VideoWriter('train.avi',cv2.VideoWriter_fourcc('P','I','M','1'), 
                          fps, (int(width), int(height)))
time.sleep(5)
while cap.isOpened():
    ret,frame = cap.read()
    
    try: 
        cv2.imshow("Training Video",frame)
        vwriter.write(frame)
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
vwriter.release()
cv2.destroyAllWindows()

## 2. Extracting coordinates

In [5]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [6]:
landmarks = ['class']
for val in range(1,34):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [7]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [8]:
with open("coords.csv",mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [9]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,
                        res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        
        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)

In [10]:
export_landmark(results,'up')

In [11]:
results.pose_landmarks.landmark

[x: 0.55341446
y: 0.534331
z: -0.8570372
visibility: 0.9998682
, x: 0.5781118
y: 0.45949486
z: -0.82318705
visibility: 0.9997402
, x: 0.5948974
y: 0.4580888
z: -0.8230589
visibility: 0.9997444
, x: 0.6085811
y: 0.45655105
z: -0.8232478
visibility: 0.99970245
, x: 0.5217842
y: 0.4623413
z: -0.81758755
visibility: 0.9997965
, x: 0.5024455
y: 0.46271914
z: -0.81697875
visibility: 0.9998189
, x: 0.48569602
y: 0.46439296
z: -0.8172237
visibility: 0.9998274
, x: 0.6289087
y: 0.465732
z: -0.506833
visibility: 0.99975276
, x: 0.46069893
y: 0.48200706
z: -0.4608168
visibility: 0.99989295
, x: 0.5875942
y: 0.5890648
z: -0.73175865
visibility: 0.9998895
, x: 0.5158602
y: 0.5978676
z: -0.7179975
visibility: 0.99993616
, x: 0.7727617
y: 0.7722447
z: -0.29034072
visibility: 0.9990503
, x: 0.3248337
y: 0.7795938
z: -0.30973512
visibility: 0.9995002
, x: 0.8936157
y: 1.059218
z: -0.35043555
visibility: 0.20785171
, x: 0.22209787
y: 1.0927107
z: -0.37894958
visibility: 0.39292365
, x: 0.86070174
y: 1.4

In [19]:
cap = cv2.VideoCapture('train.avi')
#Initiate model
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        #recolor
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detections
        results = pose.process(image)
        
        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        k = cv2.waitKey(1)
        if k == 117:
            export_landmark(results,'up')
        if k == 100:
            export_landmark(results,'down')
        
        cv2.imshow('Raw Webcam Feed',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break;
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'


## 3. Training 

In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [21]:
df = pd.read_csv('coords.csv')

In [22]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.553414,0.534331,-0.857037,0.999868,0.578112,0.459495,-0.823187,0.999740,0.594897,...,0.278216,0.000019,0.582104,2.896028,-0.188118,0.000063,0.409083,2.891962,-0.296995,0.000027
1,up,0.298316,0.217357,-0.333526,0.999911,0.307454,0.203464,-0.314020,0.999759,0.312366,...,0.134620,0.305781,0.385195,0.937053,0.014909,0.450979,0.270425,0.942263,-0.036056,0.497464
2,down,0.365290,0.439646,-0.628573,0.998388,0.375520,0.415676,-0.624889,0.996742,0.380499,...,-0.030983,0.108412,0.445133,0.867158,-0.119507,0.190882,0.336602,0.883210,-0.153117,0.176909
3,up,0.373031,0.186182,-0.370811,0.999881,0.376678,0.173634,-0.349738,0.999689,0.380530,...,0.208197,0.398352,0.423046,0.905484,0.164315,0.514231,0.340754,0.935973,0.039783,0.591700
4,up,0.370073,0.179774,-0.381279,0.999907,0.373892,0.169214,-0.359603,0.999739,0.377747,...,0.219616,0.381198,0.422675,0.910563,0.145387,0.493635,0.344373,0.935439,0.055919,0.569865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,down,0.382603,0.522863,-0.657272,0.994672,0.390050,0.502489,-0.658703,0.992645,0.395037,...,0.531574,0.076524,0.430635,0.835276,0.494082,0.090057,0.380517,0.833779,0.450969,0.148102
230,down,0.379980,0.517127,-0.649077,0.994863,0.387949,0.496929,-0.650261,0.992716,0.393312,...,0.335543,0.076931,0.442126,0.872596,0.326280,0.087540,0.374871,0.864063,0.247853,0.146464
231,down,0.379923,0.506289,-0.635311,0.995058,0.388132,0.484840,-0.634386,0.992893,0.393554,...,0.362457,0.077138,0.437793,0.861241,0.356232,0.085230,0.370570,0.852480,0.280921,0.146020
232,down,0.376676,0.499503,-0.647576,0.994652,0.386445,0.477015,-0.640525,0.992013,0.391910,...,0.366731,0.073772,0.435850,0.841440,0.365510,0.080619,0.366391,0.830203,0.284317,0.139616


In [23]:
x = df.drop('class',axis = 1)
y = df['class']

In [24]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.553414,0.534331,-0.857037,0.999868,0.578112,0.459495,-0.823187,0.999740,0.594897,0.458089,...,0.278216,0.000019,0.582104,2.896028,-0.188118,0.000063,0.409083,2.891962,-0.296995,0.000027
1,0.298316,0.217357,-0.333526,0.999911,0.307454,0.203464,-0.314020,0.999759,0.312366,0.203748,...,0.134620,0.305781,0.385195,0.937053,0.014909,0.450979,0.270425,0.942263,-0.036056,0.497464
2,0.365290,0.439646,-0.628573,0.998388,0.375520,0.415676,-0.624889,0.996742,0.380499,0.413431,...,-0.030983,0.108412,0.445133,0.867158,-0.119507,0.190882,0.336602,0.883210,-0.153117,0.176909
3,0.373031,0.186182,-0.370811,0.999881,0.376678,0.173634,-0.349738,0.999689,0.380530,0.173480,...,0.208197,0.398352,0.423046,0.905484,0.164315,0.514231,0.340754,0.935973,0.039783,0.591700
4,0.370073,0.179774,-0.381279,0.999907,0.373892,0.169214,-0.359603,0.999739,0.377747,0.169542,...,0.219616,0.381198,0.422675,0.910563,0.145387,0.493635,0.344373,0.935439,0.055919,0.569865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.382603,0.522863,-0.657272,0.994672,0.390050,0.502489,-0.658703,0.992645,0.395037,0.499938,...,0.531574,0.076524,0.430635,0.835276,0.494082,0.090057,0.380517,0.833779,0.450969,0.148102
230,0.379980,0.517127,-0.649077,0.994863,0.387949,0.496929,-0.650261,0.992716,0.393312,0.494386,...,0.335543,0.076931,0.442126,0.872596,0.326280,0.087540,0.374871,0.864063,0.247853,0.146464
231,0.379923,0.506289,-0.635311,0.995058,0.388132,0.484840,-0.634386,0.992893,0.393554,0.482393,...,0.362457,0.077138,0.437793,0.861241,0.356232,0.085230,0.370570,0.852480,0.280921,0.146020
232,0.376676,0.499503,-0.647576,0.994652,0.386445,0.477015,-0.640525,0.992013,0.391910,0.474716,...,0.366731,0.073772,0.435850,0.841440,0.365510,0.080619,0.366391,0.830203,0.284317,0.139616


In [25]:
y

0        up
1        up
2      down
3        up
4        up
       ... 
229    down
230    down
231    down
232    down
233    down
Name: class, Length: 234, dtype: object

In [26]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state = 1234)

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [28]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [29]:
fit_model = {}
for algo , pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_model[algo] = model


In [30]:
fit_model['rc'].predict(x_test)

array(['down', 'down', 'up', 'down', 'down', 'down', 'up', 'up', 'up',
       'up', 'down', 'up', 'down', 'up', 'up', 'up', 'up', 'up', 'down',
       'down', 'up', 'up', 'down', 'up', 'down', 'down', 'up', 'up',
       'down', 'down', 'up', 'down', 'up', 'up', 'down', 'down', 'down',
       'down', 'down', 'up', 'up', 'up', 'down', 'down', 'down', 'down',
       'up', 'up', 'down', 'up', 'down', 'down', 'down', 'up', 'up',
       'down', 'down', 'up', 'up', 'up', 'down', 'down', 'up', 'up', 'up',
       'down', 'up', 'up', 'up', 'up', 'up'], dtype='<U4')

In [31]:
y_test

104    down
208    down
6        up
111    down
51     down
       ... 
93       up
12       up
142      up
126      up
25       up
Name: class, Length: 71, dtype: object

## 4. Evaluation

In [32]:
from sklearn.metrics import accuracy_score , precision_score , recall_score
import pickle

In [33]:
for algo,model in fit_model.items():
    yhat = model.predict(x_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,average = "binary",pos_label = "up"),
         recall_score(y_test.values,yhat,average = "binary",pos_label = "up"))

lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [34]:
yhat = fit_model['rf'].predict(x_test)
yhat[:10]

array(['down', 'down', 'up', 'down', 'down', 'down', 'up', 'up', 'up',
       'up'], dtype=object)

## 5. Save/Load Model using pickle

In [35]:
with open('model.pkl','wb') as f:
    pickle.dump(fit_model['rf'],f)

In [36]:
with open('model.pkl','rb') as f:
    model = pickle.load(f)

In [37]:
model.predict(x_test)

array(['down', 'down', 'up', 'down', 'down', 'down', 'up', 'up', 'up',
       'up', 'down', 'up', 'down', 'up', 'up', 'up', 'up', 'up', 'down',
       'down', 'up', 'up', 'down', 'up', 'down', 'down', 'up', 'up',
       'down', 'down', 'up', 'down', 'up', 'up', 'down', 'down', 'down',
       'down', 'down', 'up', 'up', 'up', 'down', 'down', 'down', 'down',
       'up', 'up', 'down', 'up', 'down', 'down', 'down', 'up', 'up',
       'down', 'down', 'up', 'up', 'up', 'down', 'down', 'up', 'up', 'up',
       'down', 'up', 'up', 'up', 'up', 'up'], dtype=object)